# Running Constraint Scan

In [ ]:
# | default_exp runconstraintscan

This notebook can either be run as a notebook, or using nbdev a python script is produced that can be run in the command line for easier running on e.g. a cluster. It is a notebook in the nbs directory and exported as a .py file in the mmon-gcm library.

In [ ]:
# | export

import sys

import cobra
import mmon_gcm.constraintscan  # from x import y syntax doesn't work because of nbdev export format
import pandas as pd
from pandarallel import pandarallel

In [ ]:
# | export

test = False

In [ ]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is also where to change whether the light is blue or white if you want to run in the notebook

sys.argv = ["script_name",
            "../outputs/constraint_scan/constraint_scan_results_test.csv",
            "../models/4_stage_GC.json",
            "../outputs/constraint_scan/constraints_df.csv",
            "white",
            "2"]

In [ ]:
# | export

results_path = sys.argv[1]
model_path = sys.argv[2]
constraints_csv = sys.argv[3]
light_colour = sys.argv[4]
no_cores = int(sys.argv[5])

In [ ]:
# | export
constraints = pd.read_csv(constraints_csv, index_col=[0], header=[0])
model = cobra.io.load_json_model(model_path)

**If you are running from notebook you'll want to comment out the next cell! It's here so that running nbdev tests doesn't take too long, as they test running the entire notebook. It won't be exported so running as a script will be fine**

In [ ]:
test == True
constraints = constraints.iloc[:10]  # so tests don't take too long

In [ ]:
constraints.head()

,P_abs,T_l,A_l,V_gc_ind,FqFm,R_ch,L_air,L_epidermis,Vac_frac,T,R,N_gcs,n,m,r,s,C_apo,A_closed,A_open,ATPase
0,0.917691,0.000197,1.0,8.232099e-13,0.837376,0.121362,0.363686,0.124395,0.797331,293.291212,0.08205,4.756498e+08,1.829932,0.855492,7.521823e-14,2.682773e-13,0.025900,2.654215,7.513602,7.951997
1,0.881261,0.000228,1.0,2.907333e-12,0.851060,0.068245,0.348204,0.170261,0.792009,291.698522,0.08205,7.267261e+08,2.194335,0.853219,5.213651e-14,2.848145e-13,0.027046,3.830202,8.080839,15.837257
2,0.872031,0.000178,1.0,8.784094e-13,0.816784,0.156438,0.249827,0.130194,0.780852,285.335460,0.08205,1.037300e+09,2.439880,0.901650,5.550787e-14,2.612669e-13,0.026224,2.944908,6.051611,8.357127
3,0.945186,0.000172,1.0,1.031566e-12,0.841661,0.150717,0.298125,0.213744,0.794230,292.230971,0.08205,7.231671e+08,1.651735,0.911739,5.112495e-14,2.598802e-13,0.027432,3.084081,11.954409,8.384843
4,0.873152,0.000203,1.0,1.748335e-12,0.829433,0.144234,0.342282,0.149424,0.823567,286.040989,0.08205,2.877316e+08,2.056712,0.840618,7.458150e-14,1.908111e-13,0.035754,1.926858,10.939009,7.600877


In [ ]:
# | export

pandarallel.initialize(nb_workers=no_cores, progress_bar=False)
scan_solution = constraints.parallel_apply(
    mmon_gcm.constraintscan.solve_model_for_constraint_scan, args=([model, light_colour]), axis=1)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 0
Solving model 5
Solved model 0
Solved model 5
Solving model 1
Solving model 6
Solved model 6
Solved model 1
Solving model 7
Solving model 2
Solved model 7
Solved model 2
Solving model 8
Solving model 3
Solved model 3
Solved model 8
Solving model 4
Solving model 9
Solved model 4
Solved model 9


In [ ]:
# | export

solution_df = pd.DataFrame()
for i, solution in scan_solution.items():
    # this is to catch infeasible solutions
    try:
        solution_df = solution_df.append(solution.to_frame().loc[:, "fluxes"], ignore_index=True)
    except:
        solution_df = solution_df.append(pd.Series(dtype=float), ignore_index=True)

solution_df.to_csv(results_path)

/tmp/ipykernel_744406/2280842803.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(solution.to_frame().loc[:, "fluxes"], ignore_index=True)
/tmp/ipykernel_744406/2280842803.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(solution.to_frame().loc[:, "fluxes"], ignore_index=True)
/tmp/ipykernel_744406/2280842803.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(solution.to_frame().loc[:, "fluxes"], ignore_index=True)
/tmp/ipykernel_744406/2280842803.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(s